# Explore daily pairs going up and down

- Load a preprocessed candle data containing all Uni v3 pairs on Polygon
- Explore daily negative and positive price movements, with volume filter

In [103]:
import pandas as pd

from tradingstrategy.client import Client
from tradingstrategy.chain import ChainId
from tradingstrategy.timebucket import TimeBucket

client = Client.create_jupyter_client()

# Load preprocessed candle dataset
# See prepare-polygon-momentum-candles.py   
chain_id = ChainId.polygon
time_bucket = TimeBucket.d1
exchange_slugs = ["uniswap-v3", "quickswap"]
slug_str = "-and-".join(exchange_slugs)
fpath = f"/tmp/{chain_id.get_slug()}-{slug_str}-candles-{time_bucket.value}.parquet"
all_candles_df = pd.read_parquet(fpath)

# Filter out pair ids that belong to our target dataset
exchanges = client.fetch_exchange_universe()
exchange_ids = [exchanges.get_by_chain_and_slug(ChainId.polygon, s).exchange_id for s in exchange_slugs]
pairs_df = client.fetch_pair_universe().to_pandas()
pairs_df = pairs_df.loc[pairs_df["exchange_id"].isin(exchange_ids)]

#candles=GroupedCandleUniverse(
#    candles_df,
#    index_automatically=False, # Preprocesed Parquet file   
#    fix_wick_threshold=False, # Preprocesed Parquet file   
#)

#data_universe = Universe(
#    exchanges=exchanges,
#    pairs=PandasPairUniverse(pairs_df),
#    candles=GroupedCandleUniverse(candles_df),
#)

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


### Interactive viewer mode

Set Plotly chart output mode to interactive viewing.

In [104]:
from tradeexecutor.utils.notebook import OutputMode, setup_charting_and_output

setup_charting_and_output(OutputMode.interactive, image_format="svg", max_rows=35)
#setup_charting_and_output(
#    OutputMode.static, 
#    image_format="png",
#    width=1500,
#    height=1200,    
#)

## Explore pairs



In [105]:
print(f"We have total {len(pairs_df)} pairs")

fee_groupby = pairs_df.groupby("fee")
for fee_tier, df in fee_groupby:
    print(f"Fee {fee_tier} BPS has {len(df)} pairs")

We have total 5739 pairs
Fee 1 BPS has 75 pairs
Fee 5 BPS has 100 pairs
Fee 30 BPS has 5204 pairs
Fee 100 BPS has 360 pairs


## Filter untradeable pairs

- No funny quote tokens
- No stable-stable pairs
- No 1% fee pairs

In [106]:
from tradingstrategy.pair import filter_for_stablecoins, StablecoinFilteringMode

good_quote_tokes = {"USDC", "WMATIC"}

# Remove pairs with expensive 1% fee tier
# Remove stable-stable pairs
tradeable_pairs_df = pairs_df.loc[pairs_df["fee"] <= 30]
tradeable_pairs_df = filter_for_stablecoins(tradeable_pairs_df, StablecoinFilteringMode.only_volatile_pairs)
tradeable_pairs_df = tradeable_pairs_df.loc[tradeable_pairs_df["quote_token_symbol"].isin(good_quote_tokes)]

eligible_pair_ids = tradeable_pairs_df["pair_id"]

# Narrow down candle data to pairs that are left after filtering
candles_df = all_candles_df.loc[all_candles_df["pair_id"].isin(eligible_pair_ids)]

print(f"We have {len(tradeable_pairs_df)} tradeable pairs, {len(pairs_df)} total pairs")
print(f"We have {len(candles_df):,} candles for tradeable pairs, {len(all_candles_df):,} total candles")

# Denormalise pair ticker symbols on candles DataFrame 
pairs_df["ticker"] = pairs_df.apply(lambda r: f"{r.base_token_symbol}-{r.quote_token_symbol}-{r.fee}", axis=1)
# https://stackoverflow.com/questions/62133801/copy-a-column-value-from-another-dataframe-based-on-a-condition
candles_df["timestamp"] = candles_df.index  # Index is lost in merge?
candles_df = candles_df.merge(pairs_df[["pair_id", "ticker", "exchange_slug"]], on="pair_id", how="left")
candles_df = candles_df.set_index("timestamp")

We have 3692 tradeable pairs, 5739 total pairs
We have 1,080,827 candles for tradeable pairs, 1,655,927 total candles


/var/folders/tx/50wn88yd40v2_6_7fvfr98z00000gn/T/ipykernel_87259/1417214522.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Daily up and down analysis

- Figure out number of pairs under a threshold that go up and down daily



In [107]:
import plotly.graph_objects as go
import plotly.express as px

volume_threshold = 10_000
momentun_threshold = 0.005

# Get all pair candles matching volume criteria
valid = candles_df.loc[candles_df["volume"] >= volume_threshold].sort_index() 

# Filter for momentum threshold to get rid of noise 
up = valid.loc[
    (valid["close"] - valid["open"]) / valid["open"] >= momentun_threshold
]
down = valid.loc[
    (valid["close"] - valid["open"]) / valid["open"] <= -momentun_threshold
]

up_df = pd.DataFrame({"count": up.groupby("timestamp").size(), "direction": "up"})
down_df = pd.DataFrame({"count": down.groupby("timestamp").size(), "direction": "down"})

df = pd.concat([up_df, down_df]).sort_index()
display(df)

fig = px.area(df, y="count", color="direction", title=f"Daily up/down pairs, {exchange_slug} on {chain_id.get_name()}, volume threshold {volume_threshold:,} USD, momentum threshold {momentun_threshold * 100:.02f}%")
fig.show()

count direction
timestamp                  
2020-10-11      1        up
2020-10-12      1        up
2020-10-12      1      down
2020-10-13      2      down
2020-10-14      2      down
...           ...       ...
2023-11-06     24        up
2023-11-07     14        up
2023-11-07     16      down
2023-11-08     25        up
2023-11-08      4      down

[2073 rows x 2 columns]

# Weekly up and down

- Same as above, but aggregated weekly

In [108]:
# Get all pair candles matching volume criteria
from tradingstrategy.utils.groupeduniverse import resample_candles

weekly_candles = resample_candles(candles_df, pd.Timedelta(days=7))
valid = candles_df.loc[candles_df["volume"] >= volume_threshold * 7].sort_index() 

# Filter for momentum threshold to get rid of noise 
up = valid.loc[
    (valid["close"] - valid["open"]) / valid["open"] >= momentun_threshold
]
down = valid.loc[
    (valid["close"] - valid["open"]) / valid["open"] <= -momentun_threshold
]

up_df = pd.DataFrame({"count": up.groupby("timestamp").size(), "direction": "up"})
down_df = pd.DataFrame({"count": down.groupby("timestamp").size(), "direction": "down"})

df = pd.concat([up_df, down_df]).sort_index()
# display(df)

fig = px.area(df, y="count", color="direction", title=f"Weekly up/down pairs, {exchange_slug} on {chain_id.get_name()}, volume threshold {volume_threshold:,} USD, momentum threshold {momentun_threshold * 100:.02f}%")
fig.show()

## Weekly up and down, split by DEX



In [119]:
weekly_df = valid.groupby("timestamp").size()

quickswap = pd.concat([up.loc[up["exchange_slug"] == "quickswap"], down.loc[down["exchange_slug"] == "quickswap"]])
uniswap = pd.concat([up.loc[up["exchange_slug"] == "uniswap-v3"], down.loc[down["exchange_slug"] == "uniswap-v3"]])

#df = pd.concat([up_df, down_df]).sort_index()

weekly_quickswap = pd.DataFrame({"count": quickswap.groupby("timestamp").size(), "exchange_slug": "quickswap"})
weekly_uniswap = pd.DataFrame({"count": uniswap.groupby("timestamp").size(), "exchange_slug": "uniswap-v3"})

df = pd.concat([weekly_quickswap, weekly_uniswap]).sort_index()
display(df)

fig = px.area(df, y="count", color="exchange_slug", title=f"Weekly momentum pairs {chain_id.get_name()}, volume threshold {volume_threshold:,} USD, momentum threshold {momentun_threshold * 100:.02f}%, broke down by DEX")
#fig.show()
# df = pd.concat([uniswap, quickswap]).sort_index()
#
#display(df)
#fig = px.area(df, y="count", color="direction", title=f"Weekly up/down pairs, {exchange_slug} on {chain_id.get_name()}, volume threshold {volume_threshold:,} USD, momentum threshold {momentun_threshold * 100:.02f}%")
#fig.show()

count exchange_slug
timestamp                      
2020-10-25      1     quickswap
2020-11-05      1     quickswap
2020-11-06      1     quickswap
2021-01-18      1     quickswap
2021-01-19      2     quickswap
...           ...           ...
2023-11-06      3     quickswap
2023-11-07      6     quickswap
2023-11-07      6    uniswap-v3
2023-11-08      2     quickswap
2023-11-08      5    uniswap-v3

[1482 rows x 2 columns]

## Volume analysis

Analyse volume instead of pair count.

In [110]:
volume_df = all_candles_df.resample("D").sum()
eligible_volume_df = candles_df.loc[candles_df["pair_id"].isin(eligible_pair_ids)].resample("D").sum()

print(f"All volume: {volume_df['volume'].sum():,} USD, eligible volume {eligible_volume_df['volume'].sum():,} USD")

#fig = px.line(volume_df, y="volume", title="All volume")
#fig.show()

#fig = px.line(eligible_volume_df, y="volume", title="Eligible volume")
#fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=volume_df.index, y=volume_df["volume"], mode='lines', name='All volume'))
fig.add_trace(go.Scatter(x=eligible_volume_df.index, y=eligible_volume_df["volume"], mode='lines', name='Eligible volume'))
fig.show()

All volume: 65,415,094,272.0 USD, eligible volume 46,304,780,288.0 USD


/var/folders/tx/50wn88yd40v2_6_7fvfr98z00000gn/T/ipykernel_87259/1847156780.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

